In [20]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn import svm

In [2]:
data = pd.read_csv("movie_data.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
def preproc_text(text, stemmer="l"):
    text = re.sub(r"\d+", " ", text.lower())
    for i in string.punctuation:
        text = text.replace(i," ")
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    words = [i for i in tokens if not i in stop_words]
    
    if not stemmer:
        # return " ".join(words)
        wordlist = words
    if stemmer == "l": # use lancasterstemmer
        stemmer = LancasterStemmer() 
        wordlist = [stemmer.stem(word) for word in words]
    elif stemmer == "w":
        stemmer = WordNetLemmatizer()
        wordlist = [stemmer.lemmatize(word) for word in words]
    
    return " ".join(wordlist) 
    

In [4]:
corpus = data["review"]
corpus = corpus.apply(preproc_text, None)

In [5]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus[:])

In [6]:
weights = tfidf.toarray() # the TFIDF array we generate
vocab = vectorizer.get_feature_names() # dictionary 

In [7]:
def spliter(text):
    return text.split(" ")

In [8]:
wordlist = corpus.apply(spliter)
train_X, val_X = wordlist[:5000], wordlist[5000:6000]

In [9]:
model = Word2Vec(train_X, size=500, window=10, min_count=10)

In [10]:
def generate_vec(wordlist, weight):
    dic = set(model.wv.index2word)
    feature = np.zeros(500)
    for i in range(len(vocab)):
        if vocab[i] in dic:
            feature += weight[i] * model.wv[vocab[i]]
    return feature

In [14]:
feature_vector = np.zeros(5000*500).reshape(5000, 500)
for i in range(5000):
    feature_vector[i] = generate_vec(wordlist[i], weights[i])


In [18]:
def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0

label = data["sentiment"].apply(binary_transform)
train_y = label[:5000]

In [21]:
clf = svm.SVC(C=2)
clf.fit(feature_vector, train_y)


SVC(C=2)

In [32]:
val_vector = np.zeros(1000*500).reshape(1000, 500)
for i in range(5000, 6000):
    val_vector[i-5000] = generate_vec(wordlist[i], weights[i])


label_val = label[5000:6000].values

count = 0
for i in range(1000):
    if clf.predict([val_vector[i]])[0] != label_val[i]:
        count += 1


count
    
    

127